In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb

In [3]:
dfData = pd.read_csv('/home/soledad-escobar/Descargas/Orga_datos/fiuba-trocafone-tp2-final-set/events_up_to_01062018.csv')
dfTrain = pd.read_csv('/home/soledad-escobar/Descargas/Orga_datos/fiuba-trocafone-tp2-final-set/labels_training_set.csv')
dfTest = pd.read_csv('/home/soledad-escobar/Descargas/Orga_datos/fiuba-trocafone-tp2-final-set/trocafone_kaggle_test.csv')
dfSample = pd.read_csv('/home/soledad-escobar/Descargas/Orga_datos/fiuba-trocafone-tp2-final-set/trocafone_kaggle_submit_sample_all_0.csv')

/home/soledad-escobar/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
##Crea un df con todos los usuarios
dfTest['label'] = 0
dfUsers = pd.concat([dfTrain,dfTest])
dfTest = dfTest.drop('label', axis=1)

In [5]:
#Le asigna a cada usuario su cantidad de eventos dandole mas valor si son mas recientes
def pesoTiempo(time):
    mes = int(time[6])
    dia = int(time[8:10])
    if mes < 5:
        return 1
    return 10 + dia * 0.7
    
dfPE = dfData[['person', 'event', 'timestamp']]
dfPE['puntajeTiempo'] = dfPE['timestamp'].apply(pesoTiempo)
dfPE = pd.get_dummies(dfPE, columns=['event'])
columnasEventos = dfPE.columns.values[3:]
for eventoColumna in columnasEventos:
    dfPE[eventoColumna] *= dfPE['puntajeTiempo']
dfPE = dfPE.drop(['timestamp', 'puntajeTiempo'], axis=1).groupby('person').sum()
dfUsers = dfUsers.sort_values('person').merge(dfPE, on='person')

/home/soledad-escobar/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [6]:
#Le asigna a cada usuario la region en la que mas eventos tiene
regionesMasEventos = dfData['region'].value_counts().head(9).index.tolist()
def filtrarRegiones(region):
    if pd.isnull(region):
        return 'null'
    if region not in regionesMasEventos:
        return 'Other'
    return region
    
dfPE = dfData[['person', 'region']]
dfPE['regionAcotada'] = dfPE['region'].apply(filtrarRegiones)

lDictRegiones = []
temp = dfPE.drop('region', axis=1).groupby('person')
for name, group in temp:
    regiones = {'null' : 1}
    for row in group.itertuples():
        region = row.regionAcotada
        if region == 'null':
            continue
        if region in regiones:
            regiones[region] += 1
        else:
            regiones[region] = 1
        rMax = max(regiones, key=regiones.get)
    lDictRegiones.append({'person' : name, 'region' : rMax})

dfRegiones = pd.DataFrame(lDictRegiones)
dfRegiones = pd.get_dummies(dfRegiones, columns=['region'])
dfUsers = dfUsers.merge(dfRegiones, on='person')

/home/soledad-escobar/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [7]:
#Casteo dfUsers a int32
personas = dfUsers['person']
dfUsers.drop('person', axis=1, inplace=True)
dfUsers = dfUsers.astype('int32')
dfUsers['person'] = personas

In [8]:
#Agrego columna con ultima visita al sitio
temp = dfData.groupby('person')['timestamp'].max().to_frame()
dfUsers = dfUsers.merge(temp, on='person')
dfUsers['month'] = dfUsers['timestamp'].str[5:7]
dfUsers['day'] = dfUsers['timestamp'].str[8:10]
dfUsers[['day','month']] = dfUsers[['day','month']].apply(pd.to_numeric)
dfUsers['last_visit'] = dfUsers['month']*30 + dfUsers['day']
dfUsers = dfUsers.drop(['timestamp','month','day'], axis=1)

In [9]:
#Agrego columna con ultima compra
temp = dfData.loc[dfData['event'].isin(['conversion','lead','checkout'])]\
             .groupby('person')['timestamp'].max().to_frame()
dfUsers = dfUsers.merge(temp, how='left', on='person')
dfUsers = dfUsers.fillna('000000000000000000')
dfUsers['month'] = dfUsers['timestamp'].str[5:7]
dfUsers['day'] = dfUsers['timestamp'].str[8:10]
dfUsers[['day','month']] = dfUsers[['day','month']].apply(pd.to_numeric)
dfUsers['last_conversion'] = dfUsers['month']*30 + dfUsers['day']
dfUsers = dfUsers.drop(['timestamp','month','day'], axis=1)

In [10]:
#Columna con relacion entre el uso del celular y de la computadora
def filtrarDispositivo(deviceType):
    if deviceType == 'Computer':
        return 1
    elif deviceType == 'Smartphone':
        return 0
    else:
        return 2
    
dfPE = dfData[['person', 'device_type']]
dfPE['dispositivo'] = dfPE['device_type'].apply(filtrarDispositivo)
dfPE = dfPE.loc[dfPE['dispositivo'] != 2].drop('device_type', axis=1).groupby('person').mean()
dfUsers = dfUsers.merge(dfPE, on='person', how='left').fillna(0)

/home/soledad-escobar/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


# Gradient Boosting 

In [51]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score

df_y = dfTrain['label']
df_X = dfTrain.drop('label', axis=1).merge(dfUsers, on='person').drop(['person','label'], axis=1)

GBC = GradientBoostingClassifier(random_state=23)
scores = cross_val_score(GBC , df_X, df_y, scoring="roc_auc", cv=5)
scores.mean()

0.8757998577849335

# XGBoost

In [52]:
#Pruebo con XGBoost
XGBC = xgb.XGBClassifier(learning_rate =0.075, n_estimators=95, max_depth=4, min_child_weight=6, 
                         gamma=0.3, subsample=0.8, colsample_bytree=0.8, objective= 'binary:logistic',
                         scale_pos_weight=0.8, seed = 15)

scores = cross_val_score(XGBC , df_X, df_y, scoring="roc_auc", cv=5)
scores.mean()

0.8762803326776337

# Voting Classifier

In [53]:
from sklearn.ensemble import VotingClassifier

VC = VotingClassifier(estimators=[('xgb', XGBC), ('gb', GBC)], voting='soft', weights = [1.8,0.9])
scores = cross_val_score(VC , df_X, df_y, scoring="roc_auc", cv=5)
scores.mean()

0.8773219873778414

# Agrego nuevos features

In [15]:
# Agrego columnas para separar los meses y la hora
datos = dfData.loc[:,['timestamp', 'event', 'person']]
datos['date_time'] = pd.to_datetime(datos['timestamp'], format='%Y%m%d %H:%M:%S.%f')
datos['mes'] = datos['date_time'].dt.month
datos['hora'] = datos['date_time'].dt.hour

In [16]:
#Agrego columnas para ver el momento del dia en el que se generaron las conversiones
features = datos.loc[ datos['event'] == 'conversion', ['person', 'hora']]
features['mañana'] = features['hora'].apply(lambda x: 1 if ((x >= 6) & (x < 12)) else 0)
features['tarde'] = features['hora'].apply(lambda x: 1 if ((x >= 12) & (x <20)) else 0)
features['noche'] = features['hora'].apply(lambda x: 1 if ((x >= 20) & (x <= 23)) else 0)
features['madrugada'] = features['hora'].apply(lambda x: 1 if ((x >= 0) & (x < 6)) else 0)
features = features.drop('hora', axis = 'columns').groupby('person').sum()

In [45]:
# Hago un join con los features anteriores
df_feat = pd.merge(dfUsers, features, on = 'person', how = 'left').fillna(0.0).drop('label', axis = 'columns')
# Agrego la cantidad de eventos que generaron las personas
df_feat['total_events'] = (np.sum(df_feat.filter(items = ['event_ad campaign hit','event_brand listing', 'event_checkout', 
                                                         'event_conversion', 'event_generic listing', 'event_lead', 'event_search engine hit', 
                                                         'event_searched products', 'event_staticpage', 'event_viewed product','event_visited site']), 
    axis = 'columns'))

# XGBoost Classifier

In [25]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

# Me quedo solo con los datos de las pñersonas que tienen labels
df_features = df_feat.merge(dfTrain, on = 'person')

y = df_features['label']
X = df_features.drop(['person', 'label'], axis=1)

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size = 0.2)

Realizo las predicciones y luego calculo el score

In [26]:
XGBC.fit(X_train, y_train)
preds = XGBC.predict_proba(X_test)[:,1]
preds

array([0.0061182 , 0.00419453, 0.02128958, ..., 0.02706209, 0.26649815,
       0.0039666 ], dtype=float32)

In [27]:
scores = cross_val_score(XGBC , X, y, scoring = "roc_auc", cv = 5)
scores.mean()

0.8723067685424042

Como el score bajó veo como mejorar mis features.

In [28]:
# Veo en que momento del día se realizan más compras
ranking = {'mañana': df_feat['mañana'].sum(), 'tarde': df_feat['tarde'].sum(), 'noche': df_feat['noche'].sum(), 'madrugada': df_feat['madrugada'].sum()}
ranking

{'mañana': 463.0, 'tarde': 3400.0, 'noche': 1755.0, 'madrugada': 1473.0}

In [29]:
#Agrego una columna para calcular la cantidad de conversiones realizadas sobre el total de eventos relaizados
df_feat['posible_conversion'] = (df_feat['event_conversion']/ df_features['total_events'])

In [30]:
# Me quedo solo con las personas que tienen labels
df_features_1 = df_feat.merge(dfTrain, on = 'person')

y1 = df_features_1['label']
X1 = df_features_1.drop(['person', 'label'], axis=1)

scores = cross_val_score(XGBC , X1, y1, scoring = "roc_auc", cv = 5)
scores.mean()

0.8719796203099616

Como en el ranking realizado se ve en que momentos del día se realizan más conversiones, creo columnas para asignar más peso a las conversiones "más posibles" segun el momento del dia en el que ingresa el usuario

In [31]:
df_feat['posib_conv_mañ'] = (df_feat['posible_conversion'] + df_feat['mañana'])
df_feat['posib_conv_tar'] =(df_feat['posible_conversion'] + df_feat['tarde'])
df_feat['posib_conv_noch'] = (df_feat['posible_conversion'] + df_feat['noche'])
df_feat['posib_conv_mad'] = (df_feat['posible_conversion']+ df_feat['madrugada'])

In [32]:
# Me quedo solo con las personas que tienen labels
df_features_2 = df_feat.merge(dfTrain, on = 'person')

y2 = df_features_2['label']
X2 = df_features_2.drop(['person', 'label'], axis=1)

scores = cross_val_score(XGBC , X2, y2, scoring = "roc_auc", cv = 5)
scores.mean()

0.872438622363557

Como los nuevos features solo bajan aun más la precisión del algoritmo, los descarto

In [33]:
# Agrego la relación entre checkouts y conversiones
df_feat['conversion/checkout'] = (df_feat['event_conversion'] / df_feat['event_checkout'])

In [34]:
# Descarto los features anteriores
new_df_features = df_feat.drop(['posib_conv_mañ', 'posib_conv_tar', 'posib_conv_noch','posib_conv_mad'], axis = 1)

In [35]:
# Me quedo solo con las personas que tienen labels
df_features_3 = new_df_features.merge(dfTrain, on = 'person')

y3 = df_features_3['label']
X3 = df_features_3.drop(['person', 'label'], axis=1)

scores = cross_val_score(XGBC , X3, y3, scoring = "roc_auc", cv = 5)
scores.mean()

0.8740131350505322

## Agrego más features sobre marcas y modelos de los productos

Como se tienen muchos modelos diferentes pero corresponden a una cantidad acotada de marcas, trabajo sobre las marcas para crear nuevos features. Creo una columna por cada marca existente y para cada persona coloco el promedio de compra para esa marca.

In [36]:
marcas = dfData.loc[:, ['model', 'person']]
marcas['Samsung'] = marcas['model'].apply(lambda x: 1 if ('samsung' in str(x).lower()) else 0)
marcas['iPhone'] = marcas['model'].apply(lambda x: 1 if ('iphone' in str(x).lower()) else 0)
marcas['Motorola'] = marcas['model'].apply(lambda x: 1 if ('moto' in str(x).lower()) else 0)
marcas['Lenovo'] = marcas['model'].apply(lambda x: 1 if ('lenovo' in str(x).lower()) else 0)
marcas['LG'] = marcas['model'].apply(lambda x: 1 if ('lg' in str(x).lower()) else 0)
marcas['Sony'] = marcas['model'].apply(lambda x: 1 if ('sony' in str(x).lower()) else 0)
marcas['Asus'] = marcas['model'].apply(lambda x: 1 if ('asus' in str(x).lower()) else 0)
marcas['iPad'] = marcas['model'].apply(lambda x: 1 if ('ipad' in str(x).lower()) else 0)
marcas['Quantum'] = marcas['model'].apply(lambda x: 1 if ('quantum' in str(x).lower()) else 0)
dfFeat = marcas.groupby('person').sum()

In [47]:
# Hago join con los features anteriores 
new_features = pd.merge(new_df_features, dfFeat, on = 'person', how = 'left').fillna(0.0)

## Vuelvo a correr XGBoost para ver si con los nuevos features mejoran las predicciones

In [38]:
# Me quedo solo con las personas que tienen labels
df_features_4 = new_features.merge(dfTrain, on = 'person')

y4 = df_features_4['label']
X4 = df_features_4.drop(['person', 'label'], axis=1)

In [39]:
scores = cross_val_score(XGBC , X4, y4, scoring = "roc_auc", cv = 5)
scores.mean()

0.8750625603092634

Creo nuevos features para posibilidad de compra según la marca que visite el usuario

In [40]:
#Creo columnas para las marcas existentes
compras_por_marcas = dfData.loc[dfData['event'] == 'conversion', ['model', 'person']]
compras_por_marcas['Samsung_comp'] = compras_por_marcas['model'].apply(lambda x: 1 if ('samsung' in x.lower()) else 0)
compras_por_marcas['iPhone_comp'] = compras_por_marcas['model'].apply(lambda x: 1 if ('iphone' in x.lower()) else 0)
compras_por_marcas['Motorola_comp'] = compras_por_marcas['model'].apply(lambda x: 1 if ('moto' in x.lower()) else 0)
compras_por_marcas['Lenovo_comp'] = compras_por_marcas['model'].apply(lambda x: 1 if ('lenovo' in x.lower()) else 0)
compras_por_marcas['LG_comp'] = compras_por_marcas['model'].apply(lambda x: 1 if ('lg' in x.lower()) else 0)
compras_por_marcas['Sony_comp'] = compras_por_marcas['model'].apply(lambda x: 1 if ('sony' in x.lower()) else 0)
compras_por_marcas['Asus_comp'] = compras_por_marcas['model'].apply(lambda x: 1 if ('asus' in x.lower()) else 0)
compras_por_marcas['iPad_comp'] = compras_por_marcas['model'].apply(lambda x: 1 if ('iPad' in x.lower()) else 0)
compras_por_marcas['Quantum_comp'] = compras_por_marcas['model'].apply(lambda x: 1 if ('quantum' in x.lower()) else 0)
features = compras_por_marcas.groupby('person').mean()

In [48]:
set_features = pd.merge(new_features, features, on = 'person', how = 'left').fillna(0.0)

In [42]:
# Me quedo solo con las personas que tienen labels
df_features_5 = set_features.merge(dfTrain, on = 'person')

y5 = df_features_5['label']
X5 = df_features_5.drop(['person', 'label'], axis=1)

In [43]:
scores = cross_val_score(XGBC, X5, y5, scoring = "roc_auc", cv = 5)
scores.mean()

0.8748216507702689